Imports

In [ ]:
from numpy import array, asarray
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import tensorflow as tf

load doc into memory

In [ ]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r', encoding="utf-8")
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:
# load
# !dir
in_filename = './usable_data.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

tokenization

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
print(type(sequences), type(sequences[0]), type(sequences[0][0]))

In [ ]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Sequences

In [ ]:
# separate into input and output
# sequences = array(sequences)
X, y = sequences[:][:-1], sequences[:][-1]
# X = sequences[:][:-1]
# y = sequences[:][-1]
# X = [ l[:-1] if len(l) != 0 else 0 for l in sequences]
# y = [ l[-1] if len(l) != 0 else 0 for l in sequences]

# X = array(X, dtype=object)
# y = array(y)
# print(X)
# print(y)
y = to_categorical(y, num_classes=vocab_size)
seq_length = 50 # X.shape[1]
# print(seq_length)

Define model

In [31]:
model = tf.keras.Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            1518600   
                                                                 
 lstm_4 (LSTM)               (None, 50, 100)           60400     
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 30372)             3067572   
                                                                 
Total params: 4,737,072
Trainable params: 4,737,072
Non-trainable params: 0
_________________________________________________________________
None


compile model

In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 50) <dtype: 'float32'>
(None, 30372) <dtype: 'float32'>
embedding_2 (None, 50) float32
lstm_4 (None, 50, 50) float32
lstm_5 (None, 50, 100) float32
dense_4 (None, 100) float32
dense_5 (None, 100) float32


[None, None, None, None, None]

Fit model

In [35]:
# print(X.shape)
# re_x = array(X)
model.fit(X, tf.convert_to_tensor(y, dtype=tf.float64), batch_size=128, epochs=100, verbose=1)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})', "(<class 'list'> containing values of types set())"}), <class 'tensorflow.python.framework.ops.EagerTensor'>

save the model to file

In [ ]:
model.save('model.h5')

save the tokenizer

In [ ]:
dump(tokenizer, open('tokenizer.pkl', 'wb'))